In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C, Matern

from sklearn.decomposition import PCA

from scipy.stats import zscore

np.random.seed(1)

combo = pd.read_csv('2016-2017_combo.csv')

df = pd.read_excel('cts.xlsx', sheet_name='2016-2017')
numeric_cols = df.select_dtypes(include=[np.number]).columns
df[numeric_cols] = df[numeric_cols].apply(zscore)

In [2]:
Xs = []
ys = []

for idx, row in combo.iterrows():
    winner = row['Winner']
    loser = row['Loser']

    w_v = np.array(df[df['Team Name'] == winner])[0][1:]
    l_v = np.array(df[df['Team Name'] == loser])[0][1:]

    Xs.append(np.array([w_v, l_v]).flatten())
    ys.append(np.array([row['Winner Points'] - row['Loser points']]))

In [3]:
X = np.array(Xs)
y = np.array(ys)

gp = GaussianProcessRegressor(RBF(1.5))
gp.fit(X, y)

print('GP Regression R^2: ' + str(gp.score(X, y)))

coval = pd.read_csv('2015-2016_combo.csv')
val = pd.read_excel('cts.xlsx', sheet_name='2015-2016')
numeric_cols = val.select_dtypes(include=[np.number]).columns
val[numeric_cols] = val[numeric_cols].apply(zscore)

Xvs = []
yvs = []

for idx, row in coval.iterrows():
    winner = row['Winner']
    loser = row['Loser']

    try:
        w_v = np.array(val[val['Team Name'] == winner])[0][1:]
        l_v = np.array(val[val['Team Name'] == loser])[0][1:]
        Xvs.append(np.array([w_v, l_v]).flatten())
        yvs.append(np.array([row['Winner Points'], row['Loser points']]))
    except:
        print(winner)
        print(loser)

Xv = np.array(Xvs)
yv = np.array(yvs)

gp.score(Xv, yv)

GP Regression R^2: 1.0
Syracuse
MTSU
MTSU
Michigan State


ValueError: y_true and y_pred have different number of output (2!=1)